In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Hyperparameters
learning_rate = 0.0005
gamma = 0.98
buffer_limit = 50000
batch_size = 32

In [4]:
# replay buffer: 최신 5만개의 데이터(experience)를 갖고 있다가, 필요할 때마다 batch_size 만큼의 샘플을 뽑아서 제공
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit) # buffer 를 deque형 구조로 정의
        """
        deque: double-ended queue. 양방향에서 데이터를 처리할 수 있는 queue형 자료구조
               collections.deque()는 list와 비슷함
               collections.deque().append(): queue 오른쪽에 데이터를 추가
               size가 buffer_limit을 초과하면 자동으로 왼쪽(들어온지 가장 오래된) 데이터를 제거
        """

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n) # experience에서 크기가 n인 mini_batch 생성
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch: # mini_batch안에서 s, a, r, s' 각각에 대한 리스트 생성
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append([done_mask])

            return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), torch.tensor(done_mask_lst)

    def size(self): # buffer 크기 반환
        return len(self.buffer)

In [5]:
class Qnetwork(nn.Module):
    def __init__(self):
        super(Qnetwork, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x) # 마지막 layer는 Q value를 반환해야하기 때문에 양수만 반환하는 relu를 쓰면 안됨

        return x

    def sample_action(self, obs, epsilon): # epsilon-greedy policy
        out = self.forward(obs) # Q value
        coin = random.random()
        if coin < epsilon :
            return random.randint(0,1)
        else :
            return out.argmax().item()

In [2]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

In [4]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
        """
        collections.deque(): 좌우 양방향으로 데이터 출입이 가능한 자료구조.
        collections.deque().append(): 오른쪽에 데이터 추가.
        buffer_limit이 넘어가는 크기가 될 경우 왼쪽(가장 오래된) 데이터 제거
        """
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [6]:
class Qnetwork(nn.Module):
    def __init__(self):
        super(Qnetwork, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # 마지막 layer에서는 실수 전체 범위인 Q value를 반환해야하므로 양수만 반환하는 relu를 사용하면 안됨
        return x
      
    def sample_action(self, obs, epsilon): # epsilon-greedy policy
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [8]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size) # experience에서 batch_size 만큼의 (s,a,r,s') 샘플 추출

        q_out = q(s) # Q network를 이용하여 s의 Q value 계산
        q_a = q_out.gather(1,a) # Q(s,a) 계산
        """
        Q value를 구현하는 방식은 크게 2가지
        1. (s,a)를 한꺼번에 받아 Q value를 반환
        2. s를 입력받아 모든 a에 대한 Q value를 반환
        위 방식은 2번 방식(DQN 논문에서 사용한 방식)
        근본적으로 두 방식의 차이는 없음
        """

        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 학습 target을 계산하기 위해 target network 사용
        target = r + gamma * max_q_prime * done_mask # TD target
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [9]:
def main():
    env = gym.make('CartPole-v1')
    q = Qnetwork()
    q_target = Qnetwork() # target network를 따로 구성
    q_target.load_state_dict(q.state_dict()) # q network와 동일한 parameter로 초기화 
    """
    state_dict(): a dict containing parameters and persistenet buffers
    load_state_dict(state_dict()): copies parameters and bufers from state_dict
    """
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate) # target network는 학습 대상이 아니기 때문에 optimizer의 인자로 넘겨주지 않음 -> optimizer는 q network만 업데이트

    for n_epi in range(10000): # episode 10000개로 학습
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done: # episode 생성
            a = q.sample_action(torch.from_numpy(s).float(), epsilon) # Q network를 이용한 epsilon-greedy policy로 episode 생성      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            """
            reward scale이 너무 커서 100으로 나눔.
            transition : (s,a,r,s')을 memory(experience)에 저장
            """ 
            s = s_prime

            score += r
            if done:
                break # terminal state에 도달하면 종료
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
            """
            episode 1회 종료 후, experience 크기가 2000 이상인 경우 parameter update 10회 시행
            deepmind DQN논문에서는 transition 1회 발생 시 마다 batch data를 이용한 parameter update 1회 시행하는 방식
            언제, 몇번 update 하는가는 다양하게 설정할 수 있는 듯.
            
            update 한 번 할 때 batch_size = 32개의 transition 데이터 사용
            """

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict()) # print_interval 마다 1번씩만 target network 업데이트
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

In [10]:
main()

n_episode :20, score : 10.7, n_buffer : 213, eps : 7.9%
n_episode :40, score : 10.1, n_buffer : 414, eps : 7.8%
n_episode :60, score : 10.2, n_buffer : 618, eps : 7.7%
n_episode :80, score : 10.2, n_buffer : 822, eps : 7.6%
n_episode :100, score : 10.6, n_buffer : 1033, eps : 7.5%
n_episode :120, score : 9.9, n_buffer : 1231, eps : 7.4%
n_episode :140, score : 10.2, n_buffer : 1435, eps : 7.3%
n_episode :160, score : 10.1, n_buffer : 1636, eps : 7.2%
n_episode :180, score : 9.8, n_buffer : 1831, eps : 7.1%
n_episode :200, score : 10.9, n_buffer : 2050, eps : 7.0%
n_episode :220, score : 29.9, n_buffer : 2648, eps : 6.9%
n_episode :240, score : 32.2, n_buffer : 3293, eps : 6.8%
n_episode :260, score : 36.6, n_buffer : 4026, eps : 6.7%
n_episode :280, score : 44.0, n_buffer : 4907, eps : 6.6%
n_episode :300, score : 56.0, n_buffer : 6027, eps : 6.5%
n_episode :320, score : 89.8, n_buffer : 7823, eps : 6.4%
n_episode :340, score : 135.6, n_buffer : 10535, eps : 6.3%
n_episode :360, score 